In [1]:
from pathlib import Path
import json
import pandas as pd

In [2]:
BASE_PATH="/data01/wushiguang-slurm/Codes/soft-rec/___ablation_nit_movies_dvae"
FIRST_KEY="nextitnet"
SECOND_KEY="student"
METRIC_LIST=["NDCG@10", "NDCG*@10", "Recall@10", "Recall*@10"]
INDEX=['term']

In [3]:
def dir_finder(path: Path, keyword):
    for child in path.iterdir():
        if child.is_dir() and keyword in child.name:
            yield child

In [4]:
result = pd.DataFrame(columns=INDEX+METRIC_LIST)

In [5]:
for root in dir_finder(Path(BASE_PATH), FIRST_KEY):
    config = root.joinpath("config.json")
    if not config.exists():
        raise FileNotFoundError(config)
    
    value_dict = {}
    with config.open('r') as f:
        cfg = json.load(f)
    for key in INDEX:
        value_dict[key] = cfg[key]
    
    for d in dir_finder(root, SECOND_KEY):
        metrics = d.joinpath("test_metrics.json")
        if not metrics.exists():
            raise FileNotFoundError(metrics)
        
        with metrics.open('r') as f:
            mtrc = json.load(f)
        for key in METRIC_LIST:
            value_dict[key] = mtrc[key]
    
    result = result.append(value_dict, ignore_index=True)


In [6]:
# result.sort_values(by=['sample_rate', 'n_teacher'])
result

,term,NDCG@10,NDCG*@10,Recall@10,Recall*@10
0,without,0.051273,0.052772,0.084147,0.085599
1,with,0.050629,0.051969,0.084039,0.085516
